# Kill the VC - Google Colab Demo

This notebook demonstrates how to run a simplified version of the Kill the VC game in Google Colab. While the full game requires a local installation for webcam access and optimal performance, this demo will show the basic game mechanics and visual elements.

## Setup

First, let's install the required dependencies:

In [ ]:
!pip install pygame numpy opencv-python mediapipe

## Clone the Repository

Next, let's clone the Kill the VC repository to access the game assets:

In [ ]:
!git clone https://github.com/BlackBoyZeus/KillTheVc.git
%cd KillTheVc

## Demo Game Implementation

Since we can't use webcam input directly in Colab, we'll create a simplified version of the game that uses keyboard controls instead of hand gestures:

In [ ]:
%%writefile demo_game.py
import pygame
import numpy as np
import random
import sys
import os
from google.colab import output
from google.colab.output import eval_js
from IPython.display import display, Javascript

# Initialize Pygame
pygame.init()

# Set up the Pygame window
window_width, window_height = 800, 600
screen = pygame.Surface((window_width, window_height))
pygame.display.set_caption("Kill the VC - Demo")

# Initialize Pygame mixer
pygame.mixer.init()

# Load fonts
title_font = pygame.font.SysFont("Arial", 64, bold=True)
menu_font = pygame.font.SysFont("Arial", 36)
font = pygame.font.SysFont("Arial", 24)

# Game states
MENU = 0
GAME = 1
GAME_OVER = 2

# Current game state
game_state = MENU

# Load background image
try:
    background_img = pygame.image.load("assets/Assets/gringotts.jpg")
    background_img = pygame.transform.scale(background_img, (window_width, window_height))
except:
    # Create a fallback background if image not found
    background_img = pygame.Surface((window_width, window_height))
    background_img.fill((0, 0, 50))  # Dark blue background

# Load spaceship image
try:
    spaceship_img = pygame.image.load("assets/Assets/spaceship1-removebg-preview.png").convert_alpha()
    spaceship_width, spaceship_height = 100, 100
    spaceship_img = pygame.transform.scale(spaceship_img, (spaceship_width, spaceship_height))
except:
    # Create a fallback spaceship if image not found
    spaceship_img = pygame.Surface((100, 100), pygame.SRCALPHA)
    pygame.draw.polygon(spaceship_img, (200, 200, 200), [(50, 0), (0, 100), (100, 100)])

# Load laser image
try:
    laser_img = pygame.image.load("assets/Assets/laser.png").convert_alpha()
    laser_width, laser_height = 20, 40
    laser_img = pygame.transform.scale(laser_img, (laser_width, laser_height))
except:
    # Create a fallback laser if image not found
    laser_img = pygame.Surface((20, 40), pygame.SRCALPHA)
    pygame.draw.rect(laser_img, (255, 0, 0), (0, 0, 20, 40))

# Load VC image
try:
    vc_img = pygame.image.load("assets/Assets/Vc-removebg-preview.png").convert_alpha()
    vc_width, vc_height = 100, 100
    vc_img = pygame.transform.scale(vc_img, (vc_width, vc_height))
except:
    # Create a fallback VC if image not found
    vc_img = pygame.Surface((100, 100), pygame.SRCALPHA)
    pygame.draw.circle(vc_img, (150, 150, 150), (50, 50), 50)

# Game variables
spaceship_x = window_width // 2 - spaceship_width // 2
spaceship_y = window_height - spaceship_height
spaceship_speed = 5

laser_x, laser_y = None, None
laser_speed = 10
laser_state = "ready"
laser_cooldown = 0

enemies = []
num_enemies = 6
vc_index = 0
vc_health = 100
score = 0

# Health bar properties
health_bar_width, health_bar_height = 200, 20
health_bar_x, health_bar_y = window_width // 2 - health_bar_width // 2, 10

# Function to initialize enemies
def initialize_enemies():
    global enemies, vc_index
    enemies = []
    for _ in range(num_enemies):
        enemy_x = np.random.randint(0, window_width - vc_width)
        enemy_y = np.random.randint(50, window_height // 2)
        enemy_speed_x = random.uniform(-2, 2)
        enemy_speed_y = random.uniform(-1, 1)
        enemy_rotation_speed = random.uniform(-5, 5)
        enemy_health = 100
        enemies.append((enemy_x, enemy_y, enemy_speed_x, enemy_speed_y, enemy_rotation_speed, enemy_health))
    
    # Randomly select the real VC
    vc_index = random.randint(0, num_enemies - 1)

# Function to draw the health bar and score
def draw_health_bar():
    health_width = max(0, int((vc_health / 100) * health_bar_width))
    pygame.draw.rect(screen, (255, 255, 255), (health_bar_x, health_bar_y, health_bar_width, health_bar_height))
    pygame.draw.rect(screen, (0, 255, 0), (health_bar_x, health_bar_y, health_width, health_bar_height))
    health_text = font.render("VC Health", True, (255, 255, 255))
    screen.blit(health_text, (health_bar_x + health_bar_width // 2 - health_text.get_width() // 2, 
                             health_bar_y + health_bar_height + 5))

def draw_score():
    score_text = font.render("Score: " + str(score), True, (255, 255, 255))
    screen.blit(score_text, (10, 10))

# Function to draw menu
def draw_menu():
    screen.blit(background_img, (0, 0))
    
    # Draw title
    title_text = title_font.render("KILL THE VC", True, (255, 0, 0))
    screen.blit(title_text, (window_width // 2 - title_text.get_width() // 2, 100))
    
    # Draw subtitle
    subtitle_text = menu_font.render("Demo Version", True, (255, 255, 255))
    screen.blit(subtitle_text, (window_width // 2 - subtitle_text.get_width() // 2, 180))
    
    # Draw menu options
    play_text = menu_font.render("Press ENTER to Play", True, (255, 255, 255))
    screen.blit(play_text, (window_width // 2 - play_text.get_width() // 2, 280))
    
    # Draw credits
    credits_text = font.render("Created by iman, Blackboyzeus, Potus", True, (200, 200, 200))
    screen.blit(credits_text, (window_width // 2 - credits_text.get_width() // 2, window_height - 40))

# Function to start the game
def start_game():
    global game_state, vc_health, score, laser_state, laser_cooldown
    
    # Initialize game variables
    vc_health = 100
    score = 0
    laser_state = "ready"
    laser_cooldown = 0
    
    # Initialize enemies
    initialize_enemies()
    
    # Change game state
    game_state = GAME

# Function to convert pygame surface to base64 image for display in Colab
def surface_to_base64(surface):
    import base64
    from io import BytesIO
    import pygame
    
    data = pygame.image.tostring(surface, 'RGB')
    from PIL import Image
    image = Image.frombytes('RGB', surface.get_size(), data)
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode()
    return img_str

# Main game loop function
def game_loop(key_pressed=None):
    global game_state, spaceship_x, spaceship_y, laser_state, laser_x, laser_y, laser_cooldown, vc_health, score, enemies
    
    # Handle key presses
    if key_pressed:
        if game_state == MENU:
            if key_pressed == 'Enter':
                start_game()
        elif game_state == GAME:
            if key_pressed == 'ArrowLeft':
                spaceship_x -= spaceship_speed
            elif key_pressed == 'ArrowRight':
                spaceship_x += spaceship_speed
            elif key_pressed == 'ArrowUp':
                spaceship_y -= spaceship_speed
            elif key_pressed == 'ArrowDown':
                spaceship_y += spaceship_speed
            elif key_pressed == ' ' and laser_cooldown == 0:
                if laser_state == "ready":
                    laser_x = spaceship_x + spaceship_width // 2 - laser_width // 2
                    laser_y = spaceship_y
                    laser_state = "fire"
                    laser_cooldown = 20
            elif key_pressed == 'Escape':
                game_state = MENU
        elif game_state == GAME_OVER:
            if key_pressed == 'Enter' or key_pressed == ' ':
                game_state = MENU
    
    # Keep spaceship within bounds
    spaceship_x = max(0, min(spaceship_x, window_width - spaceship_width))
    spaceship_y = max(window_height // 2, min(spaceship_y, window_height - spaceship_height))
    
    # Handle different game states
    if game_state == MENU:
        draw_menu()
    
    elif game_state == GAME:
        # Draw background
        screen.blit(background_img, (0, 0))
        
        # Draw spaceship
        screen.blit(spaceship_img, (spaceship_x, spaceship_y))
        
        # Handle laser
        if laser_state == "fire":
            screen.blit(laser_img, (laser_x, laser_y))
            laser_y -= laser_speed
            if laser_y <= 0:
                laser_state = "ready"
        
        # Update and draw enemies
        for i, enemy in enumerate(enemies):
            enemy_x, enemy_y, enemy_speed_x, enemy_speed_y, enemy_rotation_speed, enemy_health = enemy
            
            # Move enemy
            enemy_x += enemy_speed_x
            enemy_y += enemy_speed_y
            
            # Bounce off walls
            if enemy_x <= 0 or enemy_x >= window_width - vc_width:
                enemy_speed_x *= -1
            if enemy_y <= 0 or enemy_y >= window_height // 2 - vc_height:
                enemy_speed_y *= -1
            
            # Occasionally change direction
            if random.randint(1, 100) <= 1:
                enemy_speed_x = random.uniform(-2, 2)
                enemy_speed_y = random.uniform(-1, 1)
            
            # Draw enemy
            if i == vc_index:
                screen.blit(vc_img, (enemy_x, enemy_y))
                # Draw a subtle indicator for the real VC
                pygame.draw.circle(screen, (255, 0, 0), 
                                  (int(enemy_x + vc_width // 2), int(enemy_y + vc_height // 2)), 
                                  int(vc_width // 2) + 5, 2)
            else:
                # Regular enemies are rotated
                rotated_enemy = pygame.transform.rotate(vc_img, (enemy_rotation_speed * pygame.time.get_ticks() / 100) % 360)
                rotated_rect = rotated_enemy.get_rect(center=(enemy_x + vc_width // 2, enemy_y + vc_height // 2))
                screen.blit(rotated_enemy, rotated_rect.topleft)
            
            # Check for laser collision
            if laser_state == "fire":
                if (laser_x >= enemy_x and laser_x <= enemy_x + vc_width and 
                    laser_y >= enemy_y and laser_y <= enemy_y + vc_height):
                    # Hit!
                    laser_state = "ready"
                    
                    if i == vc_index:
                        # Hit the real VC
                        vc_health -= 25
                        score += 25
                        # Visual feedback
                        pygame.draw.circle(screen, (255, 255, 0), 
                                          (int(enemy_x + vc_width // 2), int(enemy_y + vc_height // 2)), 
                                          50, 0)
                    else:
                        # Hit regular enemy
                        score += 5
                        # Visual feedback
                        pygame.draw.circle(screen, (255, 255, 255), 
                                          (int(enemy_x + vc_width // 2), int(enemy_y + vc_height // 2)), 
                                          30, 0)
            
            # Update enemy
            enemy = (enemy_x, enemy_y, enemy_speed_x, enemy_speed_y, enemy_rotation_speed, enemy_health)
            enemies[i] = enemy
        
        # Draw UI elements
        draw_health_bar()
        draw_score()
        
        # Check for victory
        if vc_health <= 0:
            game_state = GAME_OVER
        
        # Update laser cooldown
        if laser_cooldown > 0:
            laser_cooldown -= 1
    
    elif game_state == GAME_OVER:
        # Draw victory screen
        screen.blit(background_img, (0, 0))
        
        victory_title = title_font.render("VICTORY!", True, (255, 215, 0))
        screen.blit(victory_title, (window_width // 2 - victory_title.get_width() // 2, 100))
        
        victory_text = menu_font.render("You destroyed the VC! Welcome to IPO", True, (255, 255, 255))
        screen.blit(victory_text, (window_width // 2 - victory_text.get_width() // 2, 200))
        
        score_text = menu_font.render(f"Final Score: {score}", True, (255, 255, 255))
        screen.blit(score_text, (window_width // 2 - score_text.get_width() // 2, 260))
        
        continue_text = font.render("Press ENTER to return to menu", True, (200, 200, 200))
        screen.blit(continue_text, (window_width // 2 - continue_text.get_width() // 2, 350))
    
    # Return the base64 encoded image
    return surface_to_base64(screen)

# Initialize the game
initialize_enemies()

# Function to run the game in Colab
def run_game_in_colab():
    from IPython.display import display, HTML, Javascript
    import time
    
    # Create HTML for game display
    display(HTML("""
    <div style="text-align: center;">
        <h2>Kill the VC - Colab Demo</h2>
        <p>Use arrow keys to move, spacebar to fire</p>
        <img id="game_display" style="border: 2px solid black; width: 800px; height: 600px;">
        <div id="key_status" style="margin-top: 10px;">No key pressed</div>
    </div>
    <script>
        var lastKeyPressed = null;
        document.addEventListener('keydown', function(event) {
            lastKeyPressed = event.key;
            document.getElementById('key_status').innerText = 'Key pressed: ' + event.key;
            event.preventDefault();
        });
        function getLastKey() {
            var key = lastKeyPressed;
            lastKeyPressed = null;
            return key;
        }
    </script>
    """))
    
    # Game loop
    for _ in range(1000):  # Run for a limited number of frames
        key = eval_js('getLastKey()')
        img_data = game_loop(key)
        display(Javascript(f'''
            document.getElementById('game_display').src = 'data:image/png;base64,{img_data}';
        '''))
        time.sleep(0.05)  # Control frame rate

if __name__ == "__main__":
    run_game_in_colab()

## Run the Demo Game

Now let's run our simplified demo version of the game:

In [ ]:
!pip install pillow  # Required for image processing in the demo
%run demo_game.py

## Game Controls

- **Arrow Keys**: Move the spaceship
- **Spacebar**: Fire laser
- **Enter**: Start game / Return to menu
- **Escape**: Quit to menu

## Full Game Experience

For the full game experience with hand gesture controls, please install the game locally by following the instructions in the [GitHub repository](https://github.com/BlackBoyZeus/KillTheVc).

## About the Game

Kill the VC is a hand gesture-controlled space shooter game where you battle against a Venture Capitalist (VC) using your spaceship. The full version uses your webcam to track hand movements for controlling the ship, providing an immersive gaming experience.

### Features of the Full Game

- Hand gesture control via webcam
- Interactive menu system
- Calibration tool for hand tracking sensitivity
- Multiple enemy types
- Sound effects and background music
- Score tracking and victory screen

Enjoy the demo, and check out the full game for the complete experience!